In [1]:
''' compare GFS against alongtrack SD wind vectors at '''
import numpy as np
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import datetime
import os
import sys
from scipy import stats

In [2]:
''' SD data info '''
year = '2023'
platfs_num = ['1031','1036','1040','1041','1042','1045','1057','1064','1065','1068','1069','1083']
path_SD = os.path.expanduser('~/Documents/projects/sd-ni-wp/data_manipulate/data_merge_airsea/')
filenames_SD_all = np.sort( os.listdir(path_SD) )
filenames_SD = np.array([])
for file in filenames_SD_all:
    if ('.nc' in file) & (year in file) & (file[-7:-3] in platfs_num):
        filenames_SD = np.append(filenames_SD, file)
print(len(filenames_SD), filenames_SD)
''' GFS model data info '''
data_type = 'surface.now.'
path_model = '/Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/'
folder_by_day_model = np.sort( os.listdir(path_model) ).tolist()
print(len(folder_by_day_model), folder_by_day_model)
''' model hour/ time settings '''
hr_fcst = np.arange(1,7) # hour 1 to 6
datelim = [datetime.datetime(int(year),8,1), datetime.datetime(int(year),11,20)]
nfiles = (datelim[1]-datelim[0]).days+1
file_dates = [(datelim[0]+datetime.timedelta(hours=6*i)) for i in range(nfiles*4)]
print(file_dates)

12 ['airsea-raw-merge-2023-SD1031.nc' 'airsea-raw-merge-2023-SD1036.nc'
 'airsea-raw-merge-2023-SD1040.nc' 'airsea-raw-merge-2023-SD1041.nc'
 'airsea-raw-merge-2023-SD1042.nc' 'airsea-raw-merge-2023-SD1045.nc'
 'airsea-raw-merge-2023-SD1057.nc' 'airsea-raw-merge-2023-SD1064.nc'
 'airsea-raw-merge-2023-SD1065.nc' 'airsea-raw-merge-2023-SD1068.nc'
 'airsea-raw-merge-2023-SD1069.nc' 'airsea-raw-merge-2023-SD1083.nc']
110 ['20230801', '20230802', '20230803', '20230804', '20230805', '20230806', '20230807', '20230808', '20230809', '20230810', '20230811', '20230812', '20230813', '20230814', '20230815', '20230816', '20230817', '20230818', '20230819', '20230820', '20230821', '20230822', '20230823', '20230824', '20230825', '20230826', '20230827', '20230828', '20230829', '20230830', '20230831', '20230901', '20230904', '20230905', '20230906', '20230907', '20230908', '20230909', '20230910', '20230911', '20230912', '20230913', '20230914', '20230915', '20230916', '20230917', '20230918', '20230919', '

In [3]:
''' load all SD platf data & append specific variables to the dictionary '''
''' Wind were measured at 3.5 m for Hurricane Saildrone missions 2021-2024! '''
SDs_data = {}
vars_SD_want = ['time','longitude','latitude','UWND_MEAN','VWND_MEAN','WATER_CURRENT_SPEED_MEAN','WATER_CURRENT_DIRECTION_MEAN']
# print(SDs_data)
for i, file in enumerate( filenames_SD ):
    platf_num = file[-7:-3]
    print('start reading SD',platf_num,'nc data')
    ds = nc.Dataset(path_SD+file)
    # print(ds)
    ''' read variables '''
    for j, var_str in enumerate( vars_SD_want ):
        str_exec = var_str+' = ds.variables["'+var_str+'"][:]'
        exec(str_exec)
        if 'time' in var_str:
            time = np.array([(datetime.datetime(int(year),1,1)+datetime.timedelta(seconds=item)).timestamp() for item in time])
        # print(var_str,eval(var_str).shape)
        ''' write data to dictionary '''
        SDs_data[platf_num+'-'+var_str] = eval(var_str)
    ds.close()

start reading SD 1031 nc data
start reading SD 1036 nc data
start reading SD 1040 nc data
start reading SD 1041 nc data
start reading SD 1042 nc data
start reading SD 1045 nc data
start reading SD 1057 nc data
start reading SD 1064 nc data
start reading SD 1065 nc data
start reading SD 1068 nc data
start reading SD 1069 nc data
start reading SD 1083 nc data


In [4]:
''' loop through all the model GFS data and find the collocated values to SD alongtrack data '''
model_match_SDs_data = {}
vars_str = ['UGRD10M','VGRD10M','UFLX_AVE','VFLX_AVE']
dt_search = datetime.timedelta(minutes=0.5).total_seconds()
dx_search = 0.1
if 'lon_model' in locals():
    del lon_model, lat_model
for f, dtime_f in enumerate( file_dates ):
    folder_date = dtime_f.strftime('%Y%m%d')
    fcst_hr = dtime_f.hour
    print(f,dtime_f, folder_date, str(fcst_hr).zfill(2), )
    for hr in hr_fcst:
        path_fn = path_model+folder_date+'/reg/gfs.'+folder_date+\
            '.t'+str(fcst_hr).zfill(2)+'z.sfcf'+str(hr).zfill(3)+'.nc'
        if os.path.isfile(path_fn):
            ds = nc.Dataset(path_fn)
            # print(ds)
            if 'UGRD10M' in locals(): # clear model variables
                del UGRD10M, VGRD10M, UFLX_AVE, VFLX_AVE
            ''' read model data (only load lon/ lat once) '''
            print('Reading model time/lon/lat:',path_fn)
            if 'lon_model' not in locals():
                lon_model = ds.variables['LON'][:][0,:]
                lon_model[lon_model> 180] = lon_model[lon_model> 180]-360
                lat_model = ds.variables['LAT'][:][:,0]
                # print(lon_model.shape, np.min(lon_model),np.max(lon_model))
            timestamp_model = (dtime_f+datetime.timedelta(hours=int(hr))).timestamp()
            ''' look at one SD track at a time '''
            for s, platf_num in enumerate(platfs_num):
                ''' pull the specific SD data from dictionary '''
                # print('Reading SD',platf_num)
                timestamp_SD = SDs_data[platf_num+'-time']
                lon_SD = SDs_data[platf_num+'-longitude']
                lat_SD = SDs_data[platf_num+'-latitude']
                u_SD = SDs_data[platf_num+'-UWND_MEAN']
                v_SD = SDs_data[platf_num+'-VWND_MEAN']
                for t in range( len(timestamp_SD) ): ### loop through the SD track
                    # print(datetime.datetime.fromtimestamp(timestamp_SD[t]))
                    dt_sec = np.abs( timestamp_model-timestamp_SD[t])
                    if dt_sec< dt_search:
                        ilon_model = np.argmin( np.abs(lon_model-lon_SD[t]) )
                        ilat_model = np.argmin( np.abs(lat_model-lat_SD[t]) )
                        if (np.abs(lon_model[ilon_model]-lon_SD[t])< dx_search) & \
                            (np.abs(lat_model[ilat_model]-lat_SD[t])< dx_search):
                            if 'UGRD10M' not in locals():
                                print('Reading model u/v:',path_fn)
                                for var_str in vars_str:
                                    str_exec = var_str + '=ds.variables["'+var_str+'"][:]'
                                    exec(str_exec)
                                UGRD10M = np.squeeze(UGRD10M)
                                VGRD10M = np.squeeze(VGRD10M)
                                UFLX_AVE = np.squeeze(UFLX_AVE)
                                VFLX_AVE = np.squeeze(VFLX_AVE)
                            # print(platf_num,datetime.datetime.fromtimestamp(timestamp_SD[t]), lon_model[ilon_model],lon_SD[t],lat_model[ilat_model],lat_SD[t])
                            ''' append the collocated values in the dictionary '''
                            if platf_num+'-time' not in model_match_SDs_data:
                                model_match_SDs_data[platf_num+'-time'] = timestamp_SD[t]
                                model_match_SDs_data[platf_num+'-longitude'] = lon_SD[t]
                                model_match_SDs_data[platf_num+'-latitude'] = lat_SD[t]
                                model_match_SDs_data[platf_num+'-UFLX'] = UFLX_AVE[ilat_model,ilon_model]
                                model_match_SDs_data[platf_num+'-VFLX'] = VFLX_AVE[ilat_model,ilon_model]
                                model_match_SDs_data[platf_num+'-U10M'] = np.array([u_SD[t], UGRD10M[ilat_model,ilon_model]])
                                model_match_SDs_data[platf_num+'-V10M'] = np.array([v_SD[t], VGRD10M[ilat_model,ilon_model]])
                            else:
                                model_match_SDs_data[platf_num+'-time'] = np.append(model_match_SDs_data[platf_num+'-time'], timestamp_SD[t])
                                model_match_SDs_data[platf_num+'-longitude'] = np.append(model_match_SDs_data[platf_num+'-longitude'],lon_SD[t])
                                model_match_SDs_data[platf_num+'-latitude'] = np.append(model_match_SDs_data[platf_num+'-latitude'],lat_SD[t])
                                model_match_SDs_data[platf_num+'-UFLX'] = np.append(model_match_SDs_data[platf_num+'-UFLX'],UFLX_AVE[ilat_model,ilon_model])
                                model_match_SDs_data[platf_num+'-VFLX'] = np.append(model_match_SDs_data[platf_num+'-VFLX'],VFLX_AVE[ilat_model,ilon_model])
                                data_app = np.array([u_SD[t], UGRD10M[ilat_model,ilon_model]])
                                model_match_SDs_data[platf_num+'-U10M'] = np.column_stack( (model_match_SDs_data[platf_num+'-U10M'], data_app) )
                                data_app = np.array([v_SD[t], VGRD10M[ilat_model,ilon_model]])
                                model_match_SDs_data[platf_num+'-V10M'] = np.column_stack( (model_match_SDs_data[platf_num+'-V10M'], data_app) )
            # if platf_num+'-UFLX' in model_match_SDs_data.keys():
            #     print(platf_num,model_match_SDs_data[platf_num+'-UFLX'].size,model_match_SDs_data[platf_num+'-U10M'].shape)
            ds.close()
        else:
            print('No file:'+path_fn)
    # break 
for key in model_match_SDs_data:
    print(key, model_match_SDs_data[key].shape)

0 2023-08-01 00:00:00 20230801 00
Reading model time/lon/lat: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf001.nc
Reading model u/v: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf001.nc
Reading model time/lon/lat: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf002.nc
Reading model u/v: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf002.nc
Reading model time/lon/lat: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf003.nc
Reading model u/v: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf003.nc
Reading model time/lon/lat: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf004.nc
Reading model u/v: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/20230801/reg/gfs.20230801.t00z.sfcf004.nc
Reading model time/lon/lat: /Volumes/disk3/projects/tc2022/tc2023/GFS/sfc/

In [5]:
''' remove NaN values '''
for i, platf_num in enumerate( platfs_num ):
    isNaN = np.where( np.isnan(model_match_SDs_data[platf_num+'-U10M'][0,:]) | \
                     np.isnan(model_match_SDs_data[platf_num+'-U10M'][1,:]) )[0]
    print(platf_num, isNaN.shape, model_match_SDs_data[platf_num+'-U10M'].shape)
    if len(isNaN) > 0:
        model_match_SDs_data[platf_num+'-time'] = np.delete(model_match_SDs_data[platf_num+'-time'], isNaN)
        model_match_SDs_data[platf_num+'-longitude'] = np.delete(model_match_SDs_data[platf_num+'-longitude'], isNaN)
        model_match_SDs_data[platf_num+'-latitude'] = np.delete(model_match_SDs_data[platf_num+'-latitude'], isNaN)
        model_match_SDs_data[platf_num+'-UFLX'] = np.delete(model_match_SDs_data[platf_num+'-UFLX'], isNaN)
        model_match_SDs_data[platf_num+'-VFLX'] = np.delete(model_match_SDs_data[platf_num+'-VFLX'], isNaN)
        model_match_SDs_data[platf_num+'-U10M'] = np.delete(model_match_SDs_data[platf_num+'-U10M'], isNaN, axis=1 )
        model_match_SDs_data[platf_num+'-V10M'] = np.delete(model_match_SDs_data[platf_num+'-V10M'], isNaN, axis=1 )
        print(platf_num, model_match_SDs_data[platf_num+'-U10M'].shape)

1031 (0,) (2, 1963)
1036 (4,) (2, 2038)
1036 (2, 2034)
1040 (27,) (2, 1109)
1040 (2, 1082)
1041 (0,) (2, 1416)
1042 (3,) (2, 729)
1042 (2, 726)
1045 (20,) (2, 2062)
1045 (2, 2042)
1057 (14,) (2, 1469)
1057 (2, 1455)
1064 (32,) (2, 559)
1064 (2, 527)
1065 (0,) (2, 1007)
1068 (33,) (2, 1688)
1068 (2, 1655)
1069 (1,) (2, 2208)
1069 (2, 2207)
1083 (113,) (2, 2061)
1083 (2, 1948)


In [6]:
''' write data out to text files - by drone '''
for i, platf_num in enumerate( platfs_num ):
    vars_str_in_dict = ['time','longitude','latitude','UFLX','VFLX','U10M','V10M']
    vars_str_out = ['timestamp','longitude','latitude','UFLX(GFS) (N/m**2)','VFLX(GFS) (N/m**2)','U3.5M(SD) (m/s)','U10M(GFS) (m/s)','V3.5M(SD) (m/s)','V10M(GFS) (m/s)']
    if 'out_array' in locals():
        del out_array
    ''' column stack the values to a 2D array '''
    for j, var_str_in_dict in enumerate( vars_str_in_dict ):
        data = model_match_SDs_data[platf_num+'-'+var_str_in_dict]
        if data.ndim == 1:
            if 'out_array' not in locals():
                out_array = data
            else:
                out_array = np.column_stack( (out_array, data) )
        if data.ndim == 2:
            for k in range( data.shape[0] ):
                out_array = np.column_stack( (out_array, data[k,:]))
    print(platf_num, out_array.shape)
    ''' write 2D array to text file '''
    fn_out = 'timeseries_wnd_match_'+year+'-SD'+platf_num+'_GFS.txt'
    np.savetxt(fn_out, out_array, delimiter=' ', header=' '.join(vars_str_out))

1031 (1963, 9)
1036 (2034, 9)
1040 (1082, 9)
1041 (1416, 9)
1042 (726, 9)
1045 (2042, 9)
1057 (1455, 9)
1064 (527, 9)
1065 (1007, 9)
1068 (1655, 9)
1069 (2207, 9)
1083 (1948, 9)
